In [ ]:
pip install selenium
pip install beautifulsoup4
pip install webdriver-manager
pip install pandas

In [30]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [31]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [32]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def search_and_scrape(code):
    driver.get("https://www.aviva.co.uk/health-insurance/providers/practitioners/fee-schedule/details/SearchByText")
    time.sleep(2)

    try:
        
        input_box = driver.find_element(By.ID, "searchText")
        input_box.clear()
        input_box.send_keys(code)
        input_box.send_keys(Keys.ENTER)

        time.sleep(3)  # wait 

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Extraction 
        data_items = soup.select(".m-data-group .m-data-group-item")
        result = {"Code": code}

        for item in data_items:
            title = item.select_one(".m-data-group-item__title").text.strip()
            value = item.select_one(".m-data-group-item__data").text.strip()
            result[title] = value

        return result

    except Exception as e:
        print(f"Failed for {code}: {e}")
        return None

In [33]:
codes = ['3013', 'W3712', '3015']  
results = []

for code in codes:
    print(f"Scraping {code}...")
    result = search_and_scrape(code)
    if result:
        results.append(result)

Scraping 3013...
Scraping W3712...
Scraping 3015...


In [34]:
df = pd.DataFrame(results)
df

,Code,CCSD Code,Complexity,Surgery Fee,Anaesthetic Fee,Description
0,3013,B3013,Major Plus,£768,£303,MASTECTOMY AND IMMEDIATE RECONSTRUCTION OF BRE...
1,W3712,W3712,Major Plus,£785,£380,PRIMARY TOTAL HIP REPLACEMENT WITH OR WITHOUT ...
2,3015,B3015,Major,£384,£262,RECONSTRUCTION OF BREAST USING FIXED PROSTHESI...


In [ ]:
df.to_excel("aviva_fee_data.xlsx", index=False)
print("Saved to aviva_fee_data.xlsx")

In [35]:
driver.quit()

In [ ]:
pip install selenium 